In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys
from tqdm import tqdm

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [3]:
from underwriting import data_quality_checks

In [4]:
def cogs_analysis(df_in:pd.DataFrame,df_sales:pd.DataFrame,freq: str) -> Tuple[pd.DataFrame,pd.DataFrame]:
    df_sales['per_unit'] = df_sales['tx_total_price'] / df_sales['tx_quantity_sold']
    # set frequency
    if freq == 'monthly':
        df_sales['date'] = df_sales['sales_datetime'].dt.strftime("%Y-%m")
    elif freq == 'weekly':
        df_sales['date'] = df_sales['sales_datetime'].dt.strftime("%Y-%W")
        df_sales['week'] = df_sales['sales_datetime'].dt.strftime("%W")
    # total # of trxns
    s_total_count = df_sales.groupby('date')['tx_total_price'].count()
    df_total_count = pd.Series(s_total_count).to_frame()
    df_total_count = df_total_count.reset_index()
    df_total_count.rename(columns={'tx_total_price':'total_count'}, inplace=True)
    # revenue
    s_revenue = df_sales.groupby('date')['tx_total_price'].sum()
    df_revenue = pd.Series(s_revenue).to_frame()
    df_revenue = df_revenue.reset_index()
    df_revenue.rename(columns={'tx_total_price': 'revenue'}, inplace=True)

    df_in['per_unit_incoming'] = df_in['shipper_wholesale_price'] / df_in['shipped_quantity']
    
    # per unit price by package id
    df_in_price = df_in[df_in['shipper_wholesale_price'].notnull()]
    average_incoming_package_id = df_in_price.groupby('package_id')['per_unit_incoming'].mean()
    df_avg_incoming_price = pd.Series(average_incoming_package_id).to_frame()
    df_avg_incoming_price = df_avg_incoming_price.reset_index()
    # per unit price by product name
    average_incoming_product = df_in_price.groupby('product_name')['per_unit_incoming'].mean()
    df_avg_product = pd.Series(average_incoming_product).to_frame()
    df_avg_product = df_avg_product.reset_index()
    df_avg_product.rename(columns={'per_unit_incoming':'per_unit_product'}, inplace=True)

    # merge with (cogs by package id)
    df_cogs_package_id = pd.merge(df_sales, df_avg_incoming_price, left_on='tx_package_id', right_on='package_id', how='left')
    df_cogs_package_id['total_incoming'] = df_cogs_package_id['per_unit_incoming'] * df_cogs_package_id['tx_quantity_sold']
    df_cogs_package_id.replace([numpy.inf], numpy.nan, inplace=True)
    df_cogs_package_id_notnull = df_cogs_package_id[df_cogs_package_id['total_incoming'].notnull()]

    # sum cogs by package id
    s_cogs = df_cogs_package_id_notnull.groupby('date')['total_incoming'].sum()
    df_cogs_id = pd.Series(s_cogs).to_frame()
    df_cogs_id = df_cogs_id.reset_index()
    # count # of trxn by package id
    s_cogs_count = df_cogs_package_id_notnull.groupby('date')['total_incoming'].count()
    df_cogs_count = pd.Series(s_cogs_count).to_frame()
    df_cogs_count = df_cogs_count.reset_index()
    df_cogs_count.rename(columns={'total_incoming':'count_incoming'}, inplace=True)
    
    # merge with (cogs by product name)
    df_cogs_average_product = pd.merge(df_cogs_package_id, df_avg_product, left_on='tx_product_name', right_on='product_name', how='left')
    df_cogs_average_product['total_product'] = df_cogs_average_product['tx_quantity_sold'] * df_cogs_average_product['per_unit_product']
    df_cogs_null = df_cogs_average_product[df_cogs_average_product['per_unit_incoming'].isnull()]
    df_cogs_product = df_cogs_null[df_cogs_null['per_unit_product'].notnull()]
    # sum cogs filldown by product name
    product_sum = df_cogs_product.groupby('date')['total_product'].sum()
    df_product_sum = pd.Series(product_sum).to_frame()
    df_product_sum = df_product_sum.reset_index()
    df_product_sum.rename(columns={'total_product':'product_sum'}, inplace=True)
    # count # of trxn filldown by product name
    product_count = df_cogs_product.groupby('date')['total_product'].count()
    df_product_count = pd.Series(product_count).to_frame()
    df_product_count = df_product_count.reset_index()
    df_product_count.rename(columns={'total_product':'product_count'}, inplace=True)
    df_cogs_product_df = pd.merge(df_product_sum, df_product_count)
    
    # prepare summary
    df_summary = pd.merge(df_revenue, df_cogs_product_df, how='left')
    df_summary = pd.merge(df_summary, df_cogs_id, how='left')
    df_summary['product_sum'] = df_summary['product_sum'].fillna(0)
    df_summary['product_count'] = df_summary['product_count'].fillna(0)
    # total cogs = by product id cogs + by product name cogs
    df_summary['cogs'] = df_summary['total_incoming'] + df_summary['product_sum']
    df_summary = pd.merge(df_summary, df_cogs_count)
    df_summary = pd.merge(df_summary, df_total_count)
    # total count = by package id count + by product count
    df_summary['total_count_incoming'] = df_summary['count_incoming'] + df_summary['product_count']
    df_summary['margin_$'] = df_summary['revenue'] - df_summary['cogs']
    df_summary['margin_%'] = df_summary['margin_$'] / df_summary['revenue']
    df_summary['coverage'] = df_summary['total_count_incoming'] / df_summary['total_count']
    df_summary_simp = df_summary[['date', 'revenue', 'cogs', 'margin_$', 'margin_%', 'total_count_incoming','product_count','count_incoming', 'coverage']]
    
    return df_summary_simp,df_cogs_average_product




In [5]:
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
CURRENT_MONTH = '2022-01'

In [6]:
ca_df = pd.read_csv('CA_companies.csv',index_col = 0)
ca_df.columns = ['company_name','company_identifier','state','licenses']
ca_df['company_identifier'] = ca_df['company_identifier'].astype(str)

In [7]:
ca_df_clean = ca_df[~ca_df['company_identifier'].isin(['PL','WHT','GF','UR'])]
ca_df_clean.shape

(40, 4)

In [9]:
ca_df_clean['company_identifier'].unique()

array(['CG', 'CPA', 'CPC', 'CSC', 'CCC', 'CC', 'DGG', 'EL', 'EMA', 'EMF',
       'EMM', 'EMT', 'GT', 'GRG', 'GHC', 'HB', 'HPCC', 'HC', 'IDC', 'LBC',
       'ML', 'MW', 'MPW', 'MD', 'ND', 'PRP', 'PRL', 'RA', 'RGD', 'SFV',
       'SV', 'SLCC', 'SO', 'ST', 'TGL', 'TJR', 'TT', 'T4L', 'UHHC', 'VOY'],
      dtype=object)

In [18]:
def calculate_ca_gmv_change(company_identifier,transfer_packages_start_date,sales_transactrions_start_date,current_month):
    # setup
    revenue_change_list= list()
    df_summary_simp_list = {}
    quality_metric_list = list()
    # read data
    df_in,df_sales_deduped,bad_dl,unknown_transfer,rwp,miss_incoming,miss_receipts = data_quality_checks.run(company_identifier,transfer_packages_start_date,sales_transactrions_start_date)    
    licenses = df_sales_deduped['license_number'].unique()
    # loop thru locations
    for l in tqdm(licenses):
        print(l)
        df_in_l = df_in[df_in['license_number'] == l]
        df_sales_deduped_l = df_sales_deduped[df_sales_deduped['license_number'] == l]
        df_in_l['per_unit_incoming'] = df_in_l['shipper_wholesale_price'] / df_in_l['shipped_quantity']
        df_in_l = df_in_l[df_in_l['per_unit_incoming'] <= 10000]
        df_sales_l = df_sales_deduped_l
        df_summary_simp,df_cogs_average_product = cogs_analysis(df_in_l,df_sales_l,'monthly')
        df_summary_simp.index = df_summary_simp.date 
        # tax treatment
        df_summary_simp['revenue_after_tax'] = df_summary_simp['revenue'] * 1.15
        df_summary_simp['cogs_after_tax'] = df_summary_simp['cogs'] * 1.27
        df_summary_simp['margin_$_after_tax'] = df_summary_simp['revenue_after_tax'] - df_summary_simp['cogs_after_tax']
        df_summary_simp['margin_%_after_tax'] = df_summary_simp['margin_$_after_tax'] / df_summary_simp['revenue_after_tax']
        
        #revenue change
        df_summary_simp['revenue_change'] = df_summary_simp['revenue'].pct_change()
        df_summary_simp['revenue_change_after_tax'] = df_summary_simp['revenue_after_tax'].pct_change()
        # saving data
        if current_month in df_summary_simp['date']:
            revenue_change = df_summary_simp['revenue_change'].loc[current_month]
            revenue_change_list.append(revenue_change)
            df_summary_simp_list[l] = df_summary_simp
        else:
            revenue_change_list.append(numpy.nan)
            df_summary_simp_list[l] = df_summary_simp
            
    quality_metric_list.append(list([bad_dl,unknown_transfer,rwp,miss_incoming,miss_receipts]))       
    return revenue_change_list,df_summary_simp_list,quality_metric_list
    

In [19]:
def get_ca_gmv_change_bm_list(company_list,transfer_packages_start_date,sales_transactrions_start_date,current_month):
    gmv_change_list = list()
    cogs_df_list = {}
    quality_metric_list = {}
    for c in tqdm(company_list):
        print(c)
        gmv_change,cogs_df,quality_metric = calculate_ca_gmv_change([c],transfer_packages_start_date,sales_transactrions_start_date,current_month)
        gmv_change_list.append(gmv_change)
        cogs_df_list[c] = cogs_df
        quality_metric_list[c] = quality_metric
    return gmv_change_list,cogs_df_list,quality_metric_list
    
    

In [23]:
#test out
#a,b,c = get_ca_gmv_change_bm_list(list(ca_df_clean['company_identifier'])[0:2],TRANSFER_PACKAGES_START_DATE,SALES_TRANSACTIONS_START_DATE,CURRENT_MONTH)


In [ ]:
#b['CPA']['C9-0000348-LIC']

# run

In [ ]:
# no PL, WHT
# no GF (bad download history)
#UR: Data too big???

In [24]:
ca_gmv_change_list,ca_cogs_df_list,ca_quality_metric_list = get_ca_gmv_change_bm_list(list(ca_df_clean['company_identifier']),TRANSFER_PACKAGES_START_DATE,SALES_TRANSACTIONS_START_DATE,CURRENT_MONTH)


  0%|          | 0/40 [00:00<?, ?it/s]

CG
[SUCCESS] All download summaries look good!
[SUCCESS] No unknown transfer packages!
94.0% of incoming transfer packages have receiver wholesale price
# transactions missing incoming transfer package: 1034 (5.050801094177413%)
# transactions total: 20472
# receipts with mismatching transactions: 0 (0.0%)
# receipts missing transactions: 0 (0.0%)
# receipts total: 11263



100%|██████████| 1/1 [00:00<00:00,  5.60it/s]

C9-0000157-LIC



  2%|▎         | 1/40 [00:10<07:08, 10.99s/it]

...

C10-0000817-LIC
C9-0000082-LIC



 98%|█████████▊| 39/40 [25:26<00:31, 31.31s/it]

VOY
[SUCCESS] All download summaries look good!
[FAILURE] Fxound a total of 13231 unknown transfer packages
15.0% of incoming transfer packages have receiver wholesale price
# transactions missing incoming transfer package: 151 (3.497799397729905%)
# transactions total: 4317
# receipts with mismatching transactions: 0 (0.0%)
# receipts missing transactions: 0 (0.0%)
# receipts total: 1763



100%|██████████| 40/40 [25:51<00:00, 38.78s/it]

C10-0000802-LIC


In [29]:
ca_cogs_df_list.keys()

dict_keys(['CG', 'CPA', 'CPC', 'CSC', 'CCC', 'CC', 'DGG', 'EL', 'EMA', 'EMF', 'EMM', 'EMT', 'GT', 'GRG', 'GHC', 'HB', 'HPCC', 'HC', 'IDC', 'LBC', 'ML', 'MW', 'MPW', 'MD', 'ND', 'PRP', 'PRL', 'RA', 'RGD', 'SFV', 'SV', 'SLCC', 'SO', 'ST', 'TGL', 'TJR', 'TT', 'T4L', 'UHHC', 'VOY'])

# output data

In [44]:
df = pd.DataFrame()
for k in ca_cogs_df_list.keys():
    print(k)
    for l in ca_cogs_df_list[k].keys():
        print(l)
        data = ca_cogs_df_list[k][l]
        data['company'] = k
        data['location'] = l
        print(data.shape)
        df = df.append(data)
        #data.to_excel('ca_location_analysis/'+'analysis_' + k + '_'+ l+'.xlsx')

CG
C9-0000157-LIC
(23, 17)
CPA
C10-0000461-LIC
(25, 17)
C9-0000348-LIC
(15, 17)
CPC
C9-0000056-LIC
(19, 17)
CSC
C10-0000670-LIC
(26, 17)
C10-0000918-LIC
(3, 17)
CCC
C12-0000087-LIC
(26, 17)
CC
C10-0000758-LIC
(12, 17)
DGG
C9-0000016-LIC
(26, 17)
EL
C9-0000370-LIC
(18, 17)
EMA
C10-0000774-LIC
(14, 17)
EMF
C10-0000939-LIC
(1, 17)
EMM
C10-0000786-LIC
(12, 17)
EMT
C10-0000695-LIC
(21, 17)
GT
C9-0000385-LIC
(4, 17)
C9-0000170-LIC
(8, 17)
GRG
C10-0000169-LIC
(26, 17)
C10-0000408-LIC
(7, 17)
GHC
C10-0000414-LIC
(16, 17)
HB
C10-0000624-LIC
(26, 17)
C10-0000339-LIC
(26, 17)
C10-0000301-LIC
(20, 17)
HPCC
C10-0000064-LIC
(26, 17)
C10-0000005-LIC
(26, 17)
HC
C9-0000341-LIC
(12, 17)
IDC
C9-0000320-LIC
(23, 17)
LBC
C9-0000154-LIC
(22, 17)
ML
C9-0000300-LIC
(20, 17)
MW
C10-0000332-LIC
(26, 17)
MPW
C10-0000547-LIC
(20, 17)
MD
C9-0000247-LIC
(26, 17)
ND
C9-0000427-LIC
(8, 17)
PRP
C10-0000403-LIC
(14, 17)
C10-0000155-LIC
(14, 17)
C10-0000286-LIC
(14, 17)
PRL
C10-0000759-LIC
(14, 17)
RA
C10-0000596-LIC
(

In [46]:
df[df['company'] == 'CPA']

,date,revenue,cogs,margin_$,margin_%,total_count_incoming,product_count,count_incoming,coverage,revenue_after_tax,cogs_after_tax,margin_$_after_tax,margin_%_after_tax,revenue_change,revenue_change_after_tax,company,location
date,,,,,,,,,,,,,,,,,
2020-02,2020-02,83584.40,30243.778246,53340.621754,0.638165,1349.0,69.0,1280,0.798697,96122.0600,38409.598372,57712.461628,0.600408,NaN,NaN,CPA,C10-0000461-LIC
2020-03,2020-03,302045.80,138484.075349,163561.724651,0.541513,5767.0,153.0,5614,0.906049,347352.6700,175874.775693,171477.894307,0.493671,2.613662,2.613662,CPA,C10-0000461-LIC
2020-04,2020-04,342617.30,169160.393624,173456.906376,0.506270,6773.0,31.0,6742,0.961528,394009.8950,214833.699903,179176.195097,0.454750,0.134322,0.134322,CPA,C10-0000461-LIC
2020-05,2020-05,300627.32,136682.753726,163944.566274,0.545342,6121.0,22.0,6099,0.925881,345721.4180,173587.097232,172134.320768,0.497899,-0.122557,-0.122557,CPA,C10-0000461-LIC
2020-06,2020-06,728444.33,285081.361480,443362.968520,0.608644,14765.0,257.0,14508,0.758190,837710.9795,362053.329080,475657.650420,0.567806,1.423081,1.423081,CPA,C10-0000461-LIC
2020-07,2020-07,330008.60,165883.280414,164125.319586,0.497336,6968.0,0.0,6968,0.987808,379509.8900,210671.766126,168838.123874,0.444885,-0.546968,-0.546968,CPA,C10-0000461-LIC
2020-08,2020-08,317666.09,152148.096817,165517.993183,0.521044,7062.0,4.0,7058,0.961732,365316.0035,193228.082958,172087.920542,0.471066,-0.037401,-0.037401,CPA,C10-0000461-LIC
2020-09,2020-09,730952.79,277346.514689,453606.275311,0.620568,16306.0,3.0,16303,0.784319,840595.7085,352230.073656,488365.634844,0.580976,1.301010,1.301010,CPA,C10-0000461-LIC
2020-10,2020-10,612280.28,271582.133503,340698.146497,0.556441,15098.0,3.0,15095,0.885721,704122.3220,344909.309548,359213.012452,0.510157,-0.162353,-0.162353,CPA,C10-0000461-LIC


In [70]:
df.reset_index(drop = True)

,date,revenue,cogs,margin_$,margin_%,total_count_incoming,product_count,count_incoming,coverage,revenue_after_tax,cogs_after_tax,margin_$_after_tax,margin_%_after_tax,revenue_change,revenue_change_after_tax,company,location
0,2020-01,16467.53,7682.781465,8784.748535,0.533459,248.0,25.0,223,0.729412,18937.6595,9757.132461,9180.527039,0.484776,NaN,NaN,CG,C9-0000157-LIC
1,2020-02,25955.69,13100.733313,12854.956687,0.495265,665.0,4.0,661,0.858065,29849.0435,16637.931307,13211.112193,0.442598,0.576174,0.576174,CG,C9-0000157-LIC
2,2020-03,70721.85,38357.863697,32363.986303,0.457624,1899.0,3.0,1896,0.989062,81330.1275,48714.486896,32615.640604,0.401028,1.724715,1.724715,CG,C9-0000157-LIC
3,2020-04,39715.55,20730.950476,18984.599524,0.478014,1230.0,0.0,1230,0.998377,45672.8825,26328.307105,19344.575395,0.423546,-0.438426,-0.438426,CG,C9-0000157-LIC
4,2020-05,1397.03,636.500000,760.530000,0.544391,13.0,0.0,13,1.000000,1606.5845,808.355000,798.229500,0.496849,-0.964824,-0.964824,CG,C9-0000157-LIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,2021-10,16874.07,5047.359875,11826.710125,0.700881,534.0,181.0,353,0.862682,19405.1805,6410.147041,12995.033459,0.669668,-0.080782,-0.080782,VOY,C10-0000802-LIC
896,2021-11,18761.21,5398.519003,13362.690997,0.712251,588.0,51.0,537,0.852174,21575.3915,6856.119133,14719.272367,0.682225,0.111837,0.111837,VOY,C10-0000802-LIC
897,2021-12,32715.38,11243.897660,21471.482340,0.656312,958.0,142.0,816,0.829437,37622.6870,14279.750029,23342.936971,0.620448,0.743778,0.743778,VOY,C10-0000802-LIC
898,2022-01,24339.01,10789.363047,13549.646953,0.556705,786.0,34.0,752,0.886133,27989.8615,13702.491069,14287.370431,0.510448,-0.256038,-0.256038,VOY,C10-0000802-LIC


In [75]:
df[df['coverage'] >= 0.75]

,date,revenue,cogs,margin_$,margin_%,total_count_incoming,product_count,count_incoming,coverage,revenue_after_tax,cogs_after_tax,margin_$_after_tax,margin_%_after_tax,revenue_change,revenue_change_after_tax,company,location
date,,,,,,,,,,,,,,,,,
2020-02,2020-02,25955.69,13100.733313,12854.956687,0.495265,665.0,4.0,661,0.858065,29849.0435,16637.931307,13211.112193,0.442598,0.576174,0.576174,CG,C9-0000157-LIC
2020-03,2020-03,70721.85,38357.863697,32363.986303,0.457624,1899.0,3.0,1896,0.989062,81330.1275,48714.486896,32615.640604,0.401028,1.724715,1.724715,CG,C9-0000157-LIC
2020-04,2020-04,39715.55,20730.950476,18984.599524,0.478014,1230.0,0.0,1230,0.998377,45672.8825,26328.307105,19344.575395,0.423546,-0.438426,-0.438426,CG,C9-0000157-LIC
2020-05,2020-05,1397.03,636.500000,760.530000,0.544391,13.0,0.0,13,1.000000,1606.5845,808.355000,798.229500,0.496849,-0.964824,-0.964824,CG,C9-0000157-LIC
2020-06,2020-06,605.35,1446.900000,-841.550000,-1.390187,31.0,0.0,31,1.000000,696.1525,1837.563000,-1141.410500,-1.639598,-0.566688,-0.566688,CG,C9-0000157-LIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10,2021-10,16874.07,5047.359875,11826.710125,0.700881,534.0,181.0,353,0.862682,19405.1805,6410.147041,12995.033459,0.669668,-0.080782,-0.080782,VOY,C10-0000802-LIC
2021-11,2021-11,18761.21,5398.519003,13362.690997,0.712251,588.0,51.0,537,0.852174,21575.3915,6856.119133,14719.272367,0.682225,0.111837,0.111837,VOY,C10-0000802-LIC
2021-12,2021-12,32715.38,11243.897660,21471.482340,0.656312,958.0,142.0,816,0.829437,37622.6870,14279.750029,23342.936971,0.620448,0.743778,0.743778,VOY,C10-0000802-LIC


In [76]:
df[df['coverage'] >= 0.75].reset_index(drop = True)[['date','revenue_change']].groupby(['date']).median()

,revenue_change
date,
2020-01,NaN
2020-02,0.291432
2020-03,0.577655
2020-04,0.134322
2020-05,0.094656
2020-06,-0.016997
2020-07,0.092327
2020-08,-0.029193
2020-09,-0.044413


In [81]:
df[df['coverage'] >= 0.75].reset_index(drop = True)[['date','revenue']].groupby(['date']).count()

,revenue
date,
2020-01,4
2020-02,10
2020-03,14
2020-04,17
2020-05,19
2020-06,20
2020-07,23
2020-08,24
2020-09,24


In [47]:
#df.to_excel('ca_location_analysis/'+'ca_analysis.xlsx')

# rev change

In [54]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [55]:
rev_change = flatten_list(ca_gmv_change_list)

In [61]:
rev_change_df = pd.DataFrame(ca_gmv_change_list)
rev_change_df['company'] = ca_cogs_df_list.keys()

In [79]:
df

,date,revenue,cogs,margin_$,margin_%,total_count_incoming,product_count,count_incoming,coverage,revenue_after_tax,cogs_after_tax,margin_$_after_tax,margin_%_after_tax,revenue_change,revenue_change_after_tax,company,location
date,,,,,,,,,,,,,,,,,
2020-01,2020-01,16467.53,7682.781465,8784.748535,0.533459,248.0,25.0,223,0.729412,18937.6595,9757.132461,9180.527039,0.484776,NaN,NaN,CG,C9-0000157-LIC
2020-02,2020-02,25955.69,13100.733313,12854.956687,0.495265,665.0,4.0,661,0.858065,29849.0435,16637.931307,13211.112193,0.442598,0.576174,0.576174,CG,C9-0000157-LIC
2020-03,2020-03,70721.85,38357.863697,32363.986303,0.457624,1899.0,3.0,1896,0.989062,81330.1275,48714.486896,32615.640604,0.401028,1.724715,1.724715,CG,C9-0000157-LIC
2020-04,2020-04,39715.55,20730.950476,18984.599524,0.478014,1230.0,0.0,1230,0.998377,45672.8825,26328.307105,19344.575395,0.423546,-0.438426,-0.438426,CG,C9-0000157-LIC
2020-05,2020-05,1397.03,636.500000,760.530000,0.544391,13.0,0.0,13,1.000000,1606.5845,808.355000,798.229500,0.496849,-0.964824,-0.964824,CG,C9-0000157-LIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10,2021-10,16874.07,5047.359875,11826.710125,0.700881,534.0,181.0,353,0.862682,19405.1805,6410.147041,12995.033459,0.669668,-0.080782,-0.080782,VOY,C10-0000802-LIC
2021-11,2021-11,18761.21,5398.519003,13362.690997,0.712251,588.0,51.0,537,0.852174,21575.3915,6856.119133,14719.272367,0.682225,0.111837,0.111837,VOY,C10-0000802-LIC
2021-12,2021-12,32715.38,11243.897660,21471.482340,0.656312,958.0,142.0,816,0.829437,37622.6870,14279.750029,23342.936971,0.620448,0.743778,0.743778,VOY,C10-0000802-LIC
